In [16]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import scipy


In [104]:
def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b
    
def F(x):
    return 1/2+1/(4*math.pi)*(math.log(abs(2*x**2+2*math.sqrt(2)*x+2)/abs(abs(2*x**2-2*math.sqrt(2)*x+2)))+2*math.atan(math.sqrt(2)*x+1)+2*math.atan(math.sqrt(2)*x-1))
def inver_F(y):
    #find the solution of F(x)=y
    #Find between -1000000 and 1000000 because F(1000000)=1.0 in python
    if y>1/2:
        low =0.0
        high = 1000000.0
        mid = (high+low)/2
        while abs(high-low)>0.0000001:
            if F(mid)>y:
                high=mid
            elif F(mid)==y:
                return mid
            else: 
                low = mid
            mid = (high+low)/2
            
        return high
    if y==1/2:
        return 0
    
def CauchyNoise():
    a = random.uniform(0.5,1)
    b = inver_F(a)
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b

In [133]:
def residual(eps, beta,delta=0):
    #in our experiments, we only use delat=0 and 1e-6
    real_Q = 6001215
    if delta==0:
        alpha = eps/20
        SS = 694
                
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))

        Q = real_Q+20*SS/eps*CauchyNoise()
        #print(Q)
        indicator=0
        if real_Q<=Q+ 20*noisySS/eps*inver_F(1-beta/4) and real_Q>=Q-20*noisySS/eps*inver_F(1-beta/4):
            indicator=1

        return Q, 20*noisySS/eps*inver_F(1-beta/4),indicator
    else:
        alpha = eps/(4*math.log(1/delta))
        SS = 12998
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))
        #print(noisySS) 
        Q = real_Q+4*SS/eps*LapNoise()
        
        
        indicator=0
        if real_Q<=Q+ 4*noisySS/eps*math.log(2/beta) and real_Q>=Q-4*noisySS/eps*math.log(2/beta):
            indicator=1

        return Q, 4*noisySS/eps*math.log(2/beta),indicator

In [134]:
print("Results for Smooth based TPCH query, pure DP")
errors = []
lengths = []
correct_count = 0
num_repeat=100
for i in range(num_repeat):
    Q,length,indi = residual(eps=1, beta=0.1,delta=0)
    correct_count+=indi
    errors.append(abs(Q-6001215))
    lengths.append(length)
    
print("correct rate = " +str(correct_count/num_repeat))
avgLength = sum(lengths)/num_repeat
errors.sort()
errorQuantile = errors[int(num_repeat*0.9)]
print("Average CI length = "+str(avgLength))
print("90% error is "+ str(errorQuantile))
print("CI length/90% error = "+str(avgLength/errorQuantile))

Results for Smooth based TPCH query, pure DP
correct rate = 0.98
Average CI length = 33491.79435134671
90% error is 20720.74384841509
CI length/90% error = 1.6163413145956371


In [135]:
print("Results for Smooth based TPCH query, approximate DP")
errors = []
lengths = []
correct_count = 0
num_repeat=100
for i in range(num_repeat):
    Q,length,indi = residual(eps=1, beta=0.1,delta=0.0000001)
    correct_count+=indi
    errors.append(abs(Q-6001215))
    lengths.append(length)
    
print("correct rate = " +str(correct_count/num_repeat))
avgLength = sum(lengths)/num_repeat
errors.sort()
errorQuantile = errors[int(num_repeat*0.9)]
print("Average CI length = "+str(avgLength))
print("90% error is "+ str(errorQuantile))
print("CI length/90% error = "+str(avgLength/errorQuantile))

Results for Smooth based TPCH query, approximate DP
correct rate = 0.95
Average CI length = 172039.9366203748
90% error is 134509.13164822385
CI length/90% error = 1.2790204985510107


In [138]:
1/(4*math.log(1/0.0000001))

0.015510517210830422